# Analyse Spectractor Output and put results in fits files

- adapted from https://github.com/stubbslab/PCWG-AuxTel/blob/main/Spectractor_data_access_demo.ipynb

- author : Sylvie Dagoret-campagne
- affiliation : IJCLab/IN2P3/CNRS
- update : October 2021 8th
- DM-Stack version : **w_2021_36**
- Big CPU

Make a butler, but we don't point it at the usual repo root, we point it at the location of best reduction of the data to date.
For the current version of this, see setup_script.md

In [1]:
import matplotlib.pyplot as plt
#%matplotlib inline
# correct for bug in stack in wl_2021_02
%config IPCompleter.use_jedi=False

In [2]:
from astropy.io import fits

In [3]:
#repoDir = '/project/shared/auxTel/rerun/mfl/slurmRun/'
#repoDir='/project/shared/auxTel/rerun/dagoret'
repoDir='/project/shared/auxTel/rerun/dagoret/outputspectr_scan2021_July'

import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler(repoDir)

<ipython-input-3-a8e09fe1605e>:6: FutureWarning: Gen2 Butler has been deprecated (Butler). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  butler = dafPersist.Butler(repoDir)
<ipython-input-3-a8e09fe1605e>:6: FutureWarning: Gen2 Butler has been deprecated (LatissMapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  butler = dafPersist.Butler(repoDir)


Next, we make a NightReporter object. This is just a useful little toy project (read: not well supported, but might be helpful) to will help understand (and visualise) the night's observations. Try poking it to see what it does, some it it, _e.g._ reporter.printObsTable() is quite useful for getting a quick overview of the observations of the night.

In [4]:
from lsst.rapid.analysis.nightReport import NightReporter
reporter = NightReporter(repoDir, "2021-07-07")

Loaded data for seqNums 1 to 620


In [5]:
dayObs = '2021-07-07'

See what objects we looked at over the course of the selected dayObs, so that we can pick one that sampled a nice range of airmasses to examine further.

In [6]:
reporter.plotPerObjectAirMass()

In [7]:
objName = 'HD 160617'

In [8]:
outputfile="SpectractorOutputCollectorNCSA_HD160617,2021-07-07-holo.fits"

Here we find out from the butler which sequence numbers are observations of that star. Note that a) some will be dispersed, but some will likely be direct images or donuts, etc, and that b) of the dispersed images, some might not have run all the way through the pipeline without failure, so we're going to need to test for that.

In [9]:
seqNums = butler.queryMetadata('raw', 'seqNum', dayObs=dayObs, object=objName)

Here, for the seqNums, we ask the butler to check if the output of Spectractor actually exists, such that it can be retrieved.

In [10]:
extracted = []
for seqNum in seqNums:
    if butler.datasetExists('spectraction', dayObs=dayObs, seqNum=seqNum):
        extracted.append(seqNum)

In [11]:
print(extracted)

[234, 235, 236, 237, 239, 238, 241, 240, 242, 243, 244, 247, 248, 249, 260, 261, 262, 263, 266, 267, 268, 269, 272, 277, 278, 279, 280, 281, 282, 283, 292, 293, 298, 299, 300, 301, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 352, 353, 354, 355, 359, 361, 362, 365]


Grab a random single result just so we've got an object to poke at, such that we can use it to help us write the code that follows.

In [12]:
#all_results = []
#for seqNum in extracted:
#    results = butler.get('spectraction', dayObs=dayObs, seqNum=seqNum)
#    all_results.append(results)

In [13]:
idx=0

In [14]:
#print(all_results[idx].spectrum.lambdas)

In [15]:
#fig = plt.figure(figsize=(10,5))
#ax=fig.add_subplot(111)
#ax.plot(all_results[idx].spectrum.lambdas,all_results[idx].spectrum.data,'r')
#ax.set_title(title)

In [16]:
#result = butler.get('spectraction', dict(dayObs=dayObs, seqNum=extracted[0]))
result = butler.get('spectraction', dict(dayObs=dayObs, seqNum=317))

In [17]:
result.spectrum.header

SIMPLE  =                    T                                                  
TARGET  = 'HD 160617'          / name of the target in the image                
REDSHIFT= '0.000335'           / redshift of the target                         
GRATING = 'holo4_003'          / name of the disperser                          
ROTANGLE=  -0.1669043371015121 / [deg] angle of the dispersion axis             
D2CCD   =    178.5061134577691 / [mm] distance between disperser and CCD        
LSHIFT  =                  0.0                                                  
PARANGLE=    67.57804795567915 / parallactic angle in degree                    
TARGETX =    129.3446876015798 / target position on X axis                      
TARGETY =    490.8406546787762 / target position on Y axis                      
THETAFIT= -0.00051922423331379 / [deg] [USED] rotation angle from the Hessian an
THETAINT=                -0.28 / [deg] rotation angle interp from disperser scan
PSF_REG =    57999.999999999

In [18]:
title=result.spectrum.header['TARGET']

In [19]:
#result.spectrum?

In [20]:
fig = plt.figure(figsize=(10,5))
ax=fig.add_subplot(111)
ax.plot(result.spectrum.lambdas,result.spectrum.data,'r')
ax.set_title(title)

Text(0.5, 1.0, 'HD 160617')

In [21]:
from spectractor.extractor.spectrum import *

In [22]:
fig = plt.figure(figsize=(10,5))
ax=fig.add_subplot(111)
plot_spectrum_simple(plt.gca(), result.spectrum.lambdas, result.spectrum.data, result.spectrum.err)
ax.set_title(title)

Text(0.5, 1.0, 'HD 160617')

In [23]:
N=len(extracted)

Build a dictionary of all the results for our sequence numbers:

In [24]:
all_size=np.zeros(N)
idx=0
for seqNum in extracted:
    results = butler.get('spectraction', dayObs=dayObs, seqNum=seqNum)
    lambdas=results.spectrum.lambdas
    data=results.spectrum.data
    err=results.spectrum.err
    all_size[idx]=len(lambdas)
    idx+=1

In [25]:
all_size

array([404., 404., 405., 405., 404., 404., 405., 404., 405., 405., 299.,
       163., 458., 434., 110., 109., 107., 104., 263., 260., 259., 257.,
       416., 187., 572., 571., 569., 569., 570., 342., 200., 204., 580.,
       236., 236., 239., 580., 580., 579., 579., 579., 579., 579., 580.,
       580., 579., 579., 579., 579., 580., 378., 378., 376., 376., 402.,
       401., 400., 340.])

In [26]:
max_size=int(all_size.max())

In [27]:
output_data=np.zeros((3*N,max_size+2))

In [28]:
idx=0
for seqNum in extracted:

    results = butler.get('spectraction', dayObs=dayObs, seqNum=seqNum)
    lambdas=results.spectrum.lambdas
    data=results.spectrum.data
    err=results.spectrum.err
    siz=len(lambdas)
    output_data[3*idx+0,0]=seqNum
    output_data[3*idx+1,0]=seqNum
    output_data[3*idx+2,0]=seqNum
    output_data[3*idx+0,1]=siz
    output_data[3*idx+1,1]=siz
    output_data[3*idx+2,1]=siz
    output_data[3*idx+0,2:siz+2]=lambdas
    output_data[3*idx+1,2:siz+2]=data
    output_data[3*idx+2,2:siz+2]=err
    idx+=1

In [29]:
hdr = fits.Header()
    
#for key,value in all_my_header[idx].items():
#    hdr[str(key)]=value
        
hdu = fits.PrimaryHDU(data=output_data,header=hdr)  # with headers
#hdu = fits.PrimaryHDU(data=all_my_raw_array[idx])
    
hdul = fits.HDUList([hdu])
    
hdul.writeto(outputfile,overwrite=True)

Pick a random extraction, grab its lines, and see what their names are

In [30]:
plt.imshow(output_data,origin="lower")

In [31]:
output_data

array([[ 2.34000000e+02,  4.04000000e+02,  5.72836554e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.34000000e+02,  4.04000000e+02,  8.38258140e-11, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.34000000e+02,  4.04000000e+02,  4.38191038e-11, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 3.65000000e+02,  3.40000000e+02,  3.13553711e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.65000000e+02,  3.40000000e+02, -1.37525503e-14, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.65000000e+02,  3.40000000e+02,  1.55965691e-13, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])